In [ ]:
!pip install rasterio matplotlib

In [ ]:
patch_name = "patch1"

In [ ]:
import rasterio
import numpy as np
import sys

sys.argv = ["", "/files/private/tutorial/tiffs/" + patch_name + ".tiff", "/files/private/tutorial/tiffs/ndvi.tiff"]

input_file = sys.argv[1]
output_file = sys.argv[2]

# Open the multi-band TIFF
with rasterio.open(input_file) as src:
    red = src.read(1).astype(float)
    try:
        nir = src.read(2).astype(float)
    except IndexError:
        print("Band 2 not found, using Red as NIR for tutorial")
        nir = red.copy()
    profile = src.profile

# Scale
red = red / 10000.0
nir = nir / 10000.0

# Compute NDVI
ndvi = (nir - red) / (nir + red + 1e-6)

# Save NDVI GeoTIFF
profile.update(dtype=rasterio.float32, count=1)
with rasterio.open(output_file, "w", **profile) as dst:
    dst.write(ndvi.astype(rasterio.float32), 1)

print(f"NDVI saved to {output_file}")

In [ ]:
import rasterio
import numpy as np
import sys

sys.argv = ["", "/files/private/tutorial/tiffs/ndvi.tiff", "0.7", "/files/private/tutorial/tiffs/vegetation.tiff"]

input_file = sys.argv[1]
threshold = sys.argv[2]
output_file = sys.argv[3]

# Open NVDI file
with rasterio.open(input_file) as src:
    ndvi = src.read(1)
    profile = src.profile

# Compute mask
threshold = float(threshold)
classified = np.where(ndvi > threshold, 1, 0)

# Save make GeoTiff
profile.update(dtype=rasterio.uint8, count=1)
with rasterio.open(output_file, "w", **profile) as dst:
    dst.write(classified.astype(np.uint8), 1)

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.argv = ["", "/files/private/tutorial/tiffs/ndvi.tiff", "/files/private/tutorial/tiffs/vegetation.tiff", "/files/private/tutorial/plots/plot.png"]

ndvi_file = sys.argv[1]
vegetation_file = sys.argv[2]
output_plot = sys.argv[3]

# Read NDVI file
with rasterio.open(ndvi_file) as src:
    ndvi = src.read(1)

# Read vegetation mask
with rasterio.open(vegetation_file) as src:
    veg_mask = src.read(1)

# Compute percentage
veg_percent = (veg_mask == 1).sum() / veg_mask.size * 100
print(f"Vegetation covers {veg_percent:.2f}% of the patch")

ndvi *= 0.3 + 0.7 * veg_mask

# Plot
plt.imshow(ndvi, cmap='Greens', vmin=-0.1, vmax=1.0)
plt.title(f"Vegetation map ({veg_percent:.2f}%)")
plt.colorbar(label='NDVI')
plt.savefig(output_plot)
plt.show()
plt.close()
print(f"Plot saved to {output_plot}")